Produkty:
1. 63717975
2. 124893467
3. 108481121
4. 114228736
5. 28020568

In [1]:
import requests, os, json
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'requests'

In [ ]:
product_code = input("Podaj kod produktu: ")
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"

In [ ]:
def get_data(ancestor, selector, attribute=None, return_list=False):
     if return_list:
          return [tag.text.strip() for tag in ancestor.select(selector)] 
     if attribute:
          if selector:
               try:
                    return ancestor.select_one(selector)[attribute].strip()
               except TypeError:
                    return None
          return ancestor[attribute].strip()
     try:
          return ancestor.select_one(selector).text.strip()
     except AttributeError:
          return None

In [ ]:
selectors = {
            'opinion_id':(None, "data-entry-id"),
            'author': ("span.user-post__author-name",),
            'recommendation': ("span.user-post__author-name",),
            'stars': ("span.user-post__score-count",),
            'content': ("div.user-post__text",),
            'pros': ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
            'cons': ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
            'post_date': ("span.user-post__published > time:nth-child(1)", "datetime"),
            'purchase_date': ("span.user-post__published > time:nth-child(2)", "datetime"),
            'useful': ("button.vote-yes > span",),
            'useless': ("button.vote-no > span",)
        }

In [ ]:
all_opinions=[]
while(url):
    print(url)
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review") 
    for opinion in opinions:
        single_opinion = {
            key: get_data(opinion, *value)
                for key, value in selectors.items()
        }
    all_opinions.append(single_opinion)
    try:
        url = "https://ceneo.pl"+page.select_one("a.pagination__next")["href"]
    except TypeError:
        url = None

https://www.ceneo.pl/28020568#tab=reviews
https://ceneo.pl/28020568/opinie-2
https://ceneo.pl/28020568/opinie-3
https://ceneo.pl/28020568/opinie-4
https://ceneo.pl/28020568/opinie-5


In [ ]:

if not os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"opinions/{product_code}.json","w",encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()